# Database Cleaning
My _dbs_ are trashed. I've got 5 different database files for one project and two of them are single tables. We are going to create a single databse and merge all the data into one database, 'hsdfm_validation_data.db'. Before doing that, though, we need to get the _phantoms2_backup_ db cleaned up, beacause it alone is something of a mess.

## Phantoms Data
This _db_ was originally created under the naive paradigm of creating one table per DO measure. Since, I migrated the tables that were there at time ito tables into a single database `phantoms2.db`. `phantoms2_backup.db` should contain all the data as `phantoms2.db` plus some newer stuff. There is also some data created on seperate days in the same table of this database. Ideally, data from the first day all gets reaggregated and placed in a table 'do_testing' while data from the second day (02/07/2025) all gets aggregated into a database 'phantom_imaging_do'. In both tables, every row should have the raw data that's in the database already and a row labelling what it was measuring (which will be encoded colloquially in the table name). This labelling row should act as a foreign key to a 'master_phantoms' table that lists the makeup of each phantom. This should all be migrated into the new master study databse, 'hsdfm_validation_data.db'.
 
Adding to the confusion, `phantoms.db` is data from a two days prior (02/05/2025); a day of testing the probe (not imaging). It can probably be trashed, but for safety, we should add it to the 'do_testing' data table.

`phantoms2.db` was created from `phantoms2_backup.db` following this premise:

### Original phantom migration
We are going to refactor the tables so all the data is in one table and all the metadata (about what was recorded) is in another table. This way the names of the tables won't be such a mess and we can process things easier through SQL logic.

As it currently is we have multiple data tables that look like this:

**<name=water_yeast_25uL_of_250mg_mL_yeast>**

| id | time | time_from_start | dissolved_oxygen | nanoamperes | temperature |
|:---|:-----|:----------------|:----------------|:------------|:------------|
| 1 | datetime | 00:01 | 8.80 | 50.0 | 20.0 |
|...|...|...|...|...|...|

We want to modify the database to have two tables. One that holds all the descriptive information of the study, like this:

**<name=dissolved_oxygen_study_table>**

| id | start_time | sample_name | solvent | hemoglobin_concentration_mg_mL | microsphere_concentration_uL_mL | yeast_stock_added_uL_mL | yeast_concentration_mg_mL |
|:---|:-----------|:------------|:--------|:-------------------------------|:--------------------------------|:-------------------------|:--------------------------|
| 1 | datetime | phantom1 | water | 1 | 0.5 | 0 | 0 |
| 2 | datetime | phantom2 | water | 1 | 0.5 | 100 | 250 |
|...|...|...|...|...|...|...|...|

And a second table that holds all the data from each sample and the reference to sample data like this:

**<name=dissolved_oxygen_records>**

| id | sample_name | sample_id | time | dissolved_oxygen | nanoamperes | temperature |
|:---|:------------|:----------|:-----|:-----------------|:------------|:------------|
| 1 | phantom1 | 1 | datetime | 8.00 | 50.00 | 20.0 |
| 2 | phantom1 | 1 | datetime | 7.99 | 49.97 | 20.2 |
|...|...|...|...|...|...|...|
| 100 | phantom2 | 2 | datetime | 8.30 | 51.2 | 19.7 |
| 101 | phantom2 | 2 | datetime | 8.27 | 50.9 | 19.9 |
|...|...|...|...|...|...|...|

Where the sample_name and sample_id columns for a foreign key pair that uniquely identifies a study row in the first table.

To do migrate the current tables to this format, we will iterate through each table, prompt input for the metadata fields, and then add all the data and metadat to new tables. Then we can confirm correct migration before dropping all the old tables.


In [5]:
import sqlite3
import struct
import tkinter as tk
from tkinter import filedialog, simpledialog

In [14]:
def get_metadata_from_user(table_name):
    """Prompt user for metadata fields in a single GUI window, indicating which table/sample is being processed."""
    root = tk.Toplevel()  # Use Toplevel to ensure it's not the main loop
    root.title(f"Enter Metadata for {table_name}")
    
    tk.Label(root, text=f"Entering metadata for table: {table_name}", font=("Arial", 12, "bold")).grid(row=0, columnspan=2, pady=5)
    
    labels = [
        "Sample Name", "Solvent", "Hemoglobin Concentration (mg/mL)",
        "Microsphere Concentration (uL/mL)", "Yeast Stock Added (uL/mL)", "Yeast Concentration (mg/mL)"
    ]
    entries = []
    
    for i, label in enumerate(labels):
        tk.Label(root, text=label).grid(row=i+1, column=0)
        entry = tk.Entry(root)
        entry.grid(row=i+1, column=1)
        entries.append(entry)
    
    metadata = {}

    def submit():
        """Collect metadata and close the window."""
        metadata.update({
            "sample_name": entries[0].get(),
            "solvent": entries[1].get(),
            "hemoglobin_concentration_mg_mL": float(entries[2].get() or 0),
            "microsphere_concentration_uL_mL": float(entries[3].get() or 0),
            "yeast_stock_added_uL_mL": float(entries[4].get() or 0),
            "yeast_concentration_mg_mL": float(entries[5].get() or 0),
        })
        root.destroy()
    
    tk.Button(root, text="Submit", command=submit).grid(row=len(labels)+1, columnspan=2)
    
    root.wait_window()  # Wait for the window to close before proceeding
    return metadata


def migrate_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create new tables if they don't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dissolved_oxygen_study_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            start_time TEXT DEFAULT CURRENT_TIMESTAMP,
            sample_name TEXT,
            solvent TEXT,
            hemoglobin_concentration_mg_mL REAL,
            microsphere_concentration_uL_mL REAL,
            yeast_stock_added_uL_mL REAL,
            yeast_concentration_mg_mL REAL
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dissolved_oxygen_records (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sample_name TEXT,
            sample_id INTEGER,
            time TEXT,
            dissolved_oxygen REAL,
            nanoamperes REAL,
            temperature REAL,
            FOREIGN KEY(sample_id) REFERENCES phantom_study_metadata(id)
        )
    ''')
    conn.commit()
    
    # Get list of old tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]
    
    for table in tables:
        if table in ('dissolved_oxygen_study_table', 'dissolved_oxygen_records', 'henry_constants_calculated') or 'sqlite' in table:
            continue  # Skip new tables
        
        # Extract metadata from user
        metadata = get_metadata_from_user(table)
        # cursor.execute('''
        #     INSERT INTO phantom_study_metadata 
        #     (sample_name, solvent, hemoglobin_concentration_mg_mL, microsphere_concentration_uL_mL, 
        #     yeast_stock_added_uL_mL, yeast_concentration_mg_mL) 
        #     VALUES (?, ?, ?, ?, ?, ?)
        # ''', (metadata["sample_name"], metadata["solvent"], metadata["hemoglobin_concentration_mg_mL"],
        #       metadata["microsphere_concentration_uL_mL"], metadata["yeast_stock_added_uL_mL"],
        #       metadata["yeast_concentration_mg_mL"]))
        # 
        # sample_id = cursor.lastrowid  # Get inserted row ID
        
        # Copy data into dissolved_oxygen_records
        cursor.execute(f"SELECT id, time, dissolved_oxygen, nanoamperes, temperature FROM {table}")
        rows = cursor.fetchall()
        
        for row in rows:
            _, time, dissolved_oxygen, nanoamperes, temperature = row
            cursor.execute('''
                INSERT INTO dissolved_oxygen_records 
                (sample_name, time, dissolved_oxygen, nanoamperes, temperature) 
                VALUES (?, ?, ?, ?, ?)
            ''', (metadata["sample_name"], time, dissolved_oxygen, nanoamperes, temperature))
        
        conn.commit()
        print(f"Migrated {len(rows)} records from {table}.")
    
    # Confirm and drop old tables
    confirm = simpledialog.askstring("Confirmation", "Do you want to drop the old tables? (yes/no):")
    if confirm and confirm.lower() == "yes":
        for table in tables:
            if table not in ('dissolved_oxygen_study_table', 'dissolved_oxygen_records'):
                cursor.execute(f"DROP TABLE {table}")
                print(f"Dropped table {table}.")
        conn.commit()
    
    conn.close()
    print("Database migration complete.")

### For phantoms2_backup.db

In [4]:
# Run the migration
root = tk.Tk()
root.withdraw()  # Hide the main window
db_path = filedialog.askopenfilename(title="Select SQLite Database", filetypes=[("SQLite Database", "*.sqlite;*.db")])
if db_path:
    migrate_database(db_path)

Migrated 1604 records from water_henry.
Migrated 4200 records from water_yeast_25ul_of_250mg_mL_yeast.
Migrated 2200 records from water_yeast_100ul_of_250mg_mL_yeast.
Migrated 125 records from phantom13_14_henry.
Migrated 791 records from phantom12.
Migrated 3505 records from phantom9.
Migrated 122 records from phantom6_henry.
Migrated 631 records from phantom3_henry.
Migrated 121 records from water_for_phantom_cal.
Migrated 1002 records from phantom13.
Migrated 3215 records from phantom14.
Migrated 3263 records from phantom6.
Migrated 2628 records from phantom3.
Database migration complete.


### For phantoms.db

In [15]:
# Run the migration
root = tk.Tk()
root.withdraw()  # Hide the main window
db_path = filedialog.askopenfilename(title="Select SQLite Database", filetypes=[("SQLite Database", "*.sqlite;*.db")])
if db_path:
    migrate_database(db_path)

Migrated 129 records from water_henry_constant.
Migrated 463 records from phantom_water_cold.
Migrated 763 records from phantom13_14.
Migrated 229 records from phantom12.
Migrated 191 records from phantom9.
Migrated 235 records from phantom6.
Migrated 357 records from phantom3.
Migrated 625 records from water_w_yeast_1667ul_mL_at_2_min.
Dropped table water_henry_constant.


OperationalError: table sqlite_sequence may not be dropped

In [7]:
def split_dissolved_oxygen_records(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create new tables if they don't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS do_testing (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sample_name TEXT,
            sample_id INTEGER,
            time TEXT,
            dissolved_oxygen REAL,
            nanoamperes REAL,
            temperature REAL,
            FOREIGN KEY(sample_id) REFERENCES phantom_study_metadata(id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS do_phantom_imaging (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sample_name TEXT,
            sample_id INTEGER,
            time TEXT,
            dissolved_oxygen REAL,
            nanoamperes REAL,
            temperature REAL,
            FOREIGN KEY(sample_id) REFERENCES phantom_study_metadata(id)
        )
    ''')
    
    conn.commit()
    
    # Split records based on date
    cursor.execute("""
        INSERT INTO do_testing (sample_name, sample_id, time, dissolved_oxygen, nanoamperes, temperature)
        SELECT sample_name, sample_id, time, dissolved_oxygen, nanoamperes, temperature
        FROM dissolved_oxygen_records
        WHERE DATE(time) <= '2025-02-06'
    """
    )
    
    cursor.execute("""
        INSERT INTO do_phantom_imaging (sample_name, sample_id, time, dissolved_oxygen, nanoamperes, temperature)
        SELECT sample_name, sample_id, time, dissolved_oxygen, nanoamperes, temperature
        FROM dissolved_oxygen_records
        WHERE DATE(time) = '2025-02-07'
    """
    )
    
    conn.commit()
    conn.close()
    print("Records successfully split by date.")


In [8]:
db_path = r"C:\Users\jdivers\PycharmProjects\df_image_analysis\databases\phantoms2_backup.db"
split_dissolved_oxygen_records(db_path)

Records successfully split by date.


## MCLUT
'1d_mclut_w50000_photons.db' contains simulation parameters and results from a Monte Carlo photon simulator. It should be refactored so each row contains a label for the 'simulation_id' that is a foreign key to a 'master_mclut' database with details of the simulation, 'dimensions'and 'photon_count', (encoded in the current database name). These tables should be migrated to the new master study database.

Currently, the database has one table that looks like this: 
**<name=1d_mclut_w50000_photons.db>**

| id | mu_s | mu_a | g   | depth | transmission | reflectance | absorption |
|:---|:-----|:-----|:----|:------|:-------------|:------------|:-----------|
| 1 | BLOB | BLOB | 0.0 | 0.1   | 0.5          | 0.2         | 0.8        |
| 2 | BLOB | BLOB | 0.1 | 0.1   | 0.5          | 0.21        | 0.79       |
|...| ...  | ...  | ... | ...   | ...          | ...         | ...        |

I'd like to simply convert the BLOB dtypes to floats and add a column to that table with a foreign key that points to a table with simulation details like this:


**<name=dissolved_oxygen_records>**

| id | photon_count | dimensionality | water_n | water_mu_s | water_mu_a | tissue_n | surroundings_n |
|:---|:-------------|:---------------|:--------|:-----------|:-----------|:---------|:---------------|
| 1  | 50000 | 1.0            | 1.0 | 0.003 | 0.0 | 1.4 | 1.0            |
| ...|             | ...            | ... | ... | ...   | ... | ... | ...            | 

In [27]:
def mclut_record_update(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS mclut_simulations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    photon_count INT NOT NULL,
    dimensionality INT NOT NULL,
    water_n FLOAT NOT NULL,
    water_mu_s FLOAT NOT NULL,
    water_mu_a FLOAT NOT NULL,
    tissue_n FLOAT NOT NULL,
    surroundings_n FLOAT NOT NULL
    )""")
    
    cursor.execute("""PRAGMA foreign_keys=OFF""")
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS sim_res_temp (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    mu_s FLOAT NOT NULL,
    mu_a FLOAT NOT NULL,
    g FLOAT NOT NULL,
    depth FLOAT NOT NULL,
    transmission FLOAT NOT NULL,
    reflectance FLOAT NOT NULL,
    absorption FLOAT NOT NULL,
    simulation_id INT,
    FOREIGN KEY(simulation_id) REFERENCES mclut_simulations(id))""")
    
    cursor.execute("""
    INSERT INTO sim_res_temp (id, mu_s, mu_a, g, depth, transmission, reflectance, absorption)
    SELECT id, mu_s, mu_a, g, depth, transmission, reflectance, absorption FROM simulation_results
    """)
    
    cursor.execute("""DROP TABLE IF EXISTS simulation_results""")
    
    cursor.execute("""ALTER TABLE sim_res_temp RENAME TO simulation_results""")
    
    conn.commit()
    cursor.execute("""PRAGMA foreign_keys=ON""")
    conn.close()

In [28]:
db_path = r'C:\Users\jdivers\PycharmProjects\df_image_analysis\databases\1d_mclut_w50000_photons.db'
mclut_record_update(db_path)

In [5]:
def update_blob_to_float(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS sim_res_temp (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    mu_s FLOAT NOT NULL,
    mu_a FLOAT NOT NULL,
    g FLOAT NOT NULL,
    depth FLOAT NOT NULL,
    transmission FLOAT NOT NULL,
    reflectance FLOAT NOT NULL,
    absorption FLOAT NOT NULL,
    simulation_id INT,
    FOREIGN KEY(simulation_id) REFERENCES mclut_simulations(id))""")
    
    cursor.execute("""SELECT id, mu_s, mu_a, g, depth, transmission, reflectance, absorption FROM simulation_results""")
    data = cursor.fetchall()
    new_data = [(d[0],  struct.unpack('q', d[1])[0],  struct.unpack('q', d[2])[0], d[3], d[4], d[5], d[6], d[7]) for d in data]
 
    cursor.executemany("""INSERT INTO sim_res_temp (id, mu_s, mu_a, g, depth, transmission, reflectance, absorption)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", new_data)
    
    cursor.execute("""DROP TABLE IF EXISTS simulation_results""")
    
    cursor.execute("""ALTER TABLE sim_res_temp RENAME TO simulation_results""")
    
    conn.commit()

    conn.close()

In [6]:
db_path = r'C:\Users\jdivers\PycharmProjects\df_image_analysis\databases\1d_mclut_w50000_photons.db'
update_blob_to_float(db_path)

## Combining databases
Once we have cleaned and organized those databases/tables, I'd like to migrate all of the altered databases and 'hbo2_hb.db' into the master study database as well. It can simply be moved in as a table. No additional processing should be required.

In [3]:
conn = sqlite3.connect('hsdfm_data.db')
cursor = conn.cursor()
cursor.execute("""ATTACH DATABASE '1d_mclut_w50000_photons.db' AS mclut_db""")
cursor.execute("""CREATE TABLE IF NOT EXISTS mclut_simulations AS SELECT * FROM mclut_db.mclut_simulations""")
cursor.execute("""CREATE TABLE IF NOT EXISTS mclut AS SELECT * FROM mclut_db.simulation_results""")
cursor.execute("""UPDATE mclut SET simulation_id = 1""")
conn.commit()
cursor.execute("""DETACH DATABASE mclut_db""")
cursor.execute("""ATTACH DATABASE 'hbo2_hb.db' AS hb""")
cursor.execute("""CREATE TABLE IF NOT EXISTS hb_spectra AS SELECT * FROM hb.molar_extinction_data""")
conn.commit()
cursor.execute("""DETACH DATABASE hb""")
cursor.execute("""ATTACH DATABASE 'phantoms2_backup.db' as phantoms""")
cursor.execute("""CREATE TABLE IF NOT EXISTS phantom_imaging AS SELECT * FROM phantoms.do_phantom_imaging""")
cursor.execute("""CREATE TABLE IF NOT EXISTS do_testing AS SELECT * FROM phantoms.do_testing""")
cursor.execute("""CREATE TABLE IF NOT EXISTS phantoms AS SELECT * FROM phantoms.phantom_study_metadata""")
conn.commit()
cursor.execute("""DETACH DATABASE phantoms""")
conn.close()